
<p> <center> <a href="../Start_Here.ipynb">Home Page</a> </center> </p>

 
<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="2.1.System-Topology.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 33%; text-align: center;">
        <a href="1.Introduction-to-Distributed-Deep-Learning.ipynb">1</a>
        <a href="2.1.System-Topology.ipynb">2</a>
        <a >3</a>
        <a href="4.Convergence.ipynb">4</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="4.Convergence.ipynb">Next Notebook</a></span>
</div>

# Introduction to Distributed Deep Learning - Part 3

**Contents of this notebook:**

- [Hands-on with Distributed training](#Hands-on-with-Distributed-training)
    - [Tensorflow - Keras](#Tensorflow---Keras)
    - [Horovod](#Horovod)

**The objectives of this Notebook is to:**

- Implement distributed deep learning training using Tensorflow / Keras.
- Learn concepts of Horovod and implement them.

# Hands-on with Distributed training

This notebook will focus on **training** Deep Neural networks using **multiple GPUs** and **Data-parallelism**. The following frameworks would be used for the demonstration. 

- [Tensorflow - Keras](#Tensorflow---Keras)
- [Horovod](#Horovod)

In both cases, we will use **Single Host Multiple Device** implementations, although notes on **Multi-host multiple devices** will be presented at relevant places.

You can use either / both of the methods that we mention below. We recommend trying out both of them.

## Tensorflow - Keras


Tensorflow uses `tf.distribute.Strategy` API to distribute training across multiple GPUs, multiple machines or TPUs. Using this API, you can distribute your existing models and training code with minimal code changes. `tf.distribute.Strategy` can be used with a high-level API like **Keras**, and can also be used to distribute custom training loops.

Tensorflow Strategies can be briefly summarised into two axes : 

- **Synchronous vs asynchronous** training: 
    - In sync training, all workers train over different slices of input data in sync and aggregate gradients at each step. 
    - In async training, all workers are independently training over the input data and updating variables asynchronously. 



- **Hardware platform**: You may want to scale your training onto multiple GPUs on one machine, or multiple machines in a network (with 0 or more GPUs each), or on Cloud TPUs.

In order to support these use cases, there are 6 strategies available. Here is a short description of the strategies :

- **MirroredStrategy**
    - `tf.distribute.MirroredStrategy` supports synchronous distributed training on multiple GPUs on one machine. It creates one replica per GPU device. Each variable in the model is mirrored across all the replicas. Together, these variables form a single conceptual variable called MirroredVariable. These variables are kept in sync with each other by applying identical updates.
- **MultiWorkerMirroredStrategy**
    - `tf.distribute.MultiWorkerMirroredStrategy` is very similar to MirroredStrategy. It implements synchronous distributed training across multiple workers, each with potentially multiple GPUs. Similar to `tf.distribute.MirroredStrategy`, it creates copies of all variables in the model on each device across all workers.
    - **Note** : For multi-worker training, you need to set up the `TF_CONFIG` environment variable for each binary running in your cluster. The `TF_CONFIG` environment variable is a JSON string which specifies what tasks constitute a cluster, their addresses and each task's role in the cluster.
- **TPUStrategy**
    - `tf.distribute.TPUStrategy` lets you run your TensorFlow training on Tensor Processing Units (TPUs).
- **CentralStorageStrategy**
    - `tf.distribute.experimental.CentralStorageStrategy` does synchronous training as well. Variables are not mirrored, instead they are placed on the CPU and operations are replicated across all local GPUs. If there is only one GPU, all variables and operations will be placed on that GPU.
- **OneDeviceStrategy**
    - `tf.distribute.OneDeviceStrategy` is a strategy to place all variables and computation on a single specified device.
- **ParameterServerStrategy**
    - Parameter server training is a common data-parallel method to scale up model training on multiple machines. A parameter server training cluster consists of workers and parameter servers. Variables are created on parameter servers and they are read and updated by workers in each step.


Tensorflow by default uses [NCCL](https://developer.nvidia.com/nccl) for communication between GPUs. Kindly refer to the [System Topology Notebook](2.1.System-Topology.ipynb#NCCL) to learn more about NCCL.

Now that we've given an overview of Tensorflow and the available functionalities let us try out an example using **MirroredStrategy**. 

We will be building a small CNN and train it on the **FMNIST dataset**.

```python
#Import Necessary libraries
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import time
import os
import sys


# Set number of GPUs to use for training
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

#Print Tensorflow version
print(tf.__version__)

# Download the MNIST dataset and load it from [TensorFlow Datasets](https://www.tensorflow.org/datasets). This returns a dataset in `tf.data` format.

# Setting `with_info` to `True` includes the metadata for the entire dataset, which is being saved here to `info`.
# Among other things, this metadata object includes the number of train and test examples. 
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']
```

Let us now define a distribution strategy.

```python
### Define distribution strategy
# Create a `MirroredStrategy` object. This will handle distribution, 
# and provides a context manager (`tf.distribute.MirroredStrategy.scope`) to build your model inside.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
```

```python
# Setup input pipeline
# When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory, and tune the learning rate accordingly.
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

# Setting the batch size per GPU / replica
BATCH_SIZE_PER_REPLICA = 512
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


# Feature scaling function
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label


# Apply the scale function to the training and test data, shuffle the training data, and batching it.
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)
```

Let's create and compile the model, note that we need to create and compile the model under the context of `strategy.scope`.

We can now define callbacks to calculate `Throughput` obtained.

**Note : We use the same model used in Notebook-1 scaling efficiency experiment, that should help us calculate scaling efficiency from the available data**

```python

with strategy.scope():
  model = tf.keras.Sequential([
          tf.keras.layers.Conv2D(32, [3, 3], activation='relu',input_shape=(28, 28, 1)),
          tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
          tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
          tf.keras.layers.Dropout(0.25),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(10, activation='softmax')])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam()),
                metrics=['accuracy'])
```

```python

# ## Define the callbacks
# Callback for printing the LR at the end of each epoch.
class Throughput(tf.keras.callbacks.Callback):
    def __init__(self, total_images=0):
        self.total_images = total_images
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print('Epoch time : {}'.format(epoch_time))
        images_per_sec = round(self.total_images / epoch_time, 2)
        print('Images/sec: {}'.format(images_per_sec))
        
# Now, train the model in the usual way, calling `fit` on the model and passing in the dataset created at the beginning of the tutorial. This step is the same whether you are distributing the training or not.
model.fit(train_dataset, epochs=8, callbacks=Throughput(total_images=len(mnist_train)))

```

In [2]:
SINGULARITY_RUN="singularity run --nv --env TF_CPP_MIN_LOG_LEVEL=3  ~/DDL.simg "


In [2]:
COMMAND = SINGULARITY_RUN + ' python ../source_code/N3/tensorflow_ddl.py 2> /dev/null'
!echo $COMMAND > command && srun --partition=gpu -n1 --gres=gpu:4 /bin/bash ./command


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.

2.5.0
Dl Completed...: 100%|██████████| 4/4 [00:00<00:00,  6.43 file/s]

Dataset mnist downloaded and prepared to /home/uz6cms8/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Number of devices: 4
Epoch 1/8
8/8 [==============================] - 35s 698ms/step - loss: 1.5692 - accuracy: 0.5226
Epoch time : 34.54443669319153
Images/sec: 1736.89
Epoch 2/8
8/8 [==============================] - 0s 56ms/step - loss: 0.6142 - accuracy: 0.8047
Epoch time : 0.4413290023803711
Images/sec: 135953.0
Epoch 3/8
8/8 [==============================] - 0s 27ms/step - loss: 0.4252 - accuracy: 0.8712
Epoch time : 0.23953604698181152
Images/sec: 250484.22
Epoch 4/8
8/8 [==============================] - 0s 27ms/step - loss: 0.3371 - accuracy: 0.8982
Epoch tim

Kindly restart the kernel or run the following cell to restart the kernel to free up GPU memory before procedding to the next section.

In [3]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True) 

{'status': 'ok', 'restart': True}

## Horovod

Horovod is a distributed deep learning training framework. It is available for TensorFlow, Keras, PyTorch, and Apache MXNet.

Horovod is an open-source tool initially developed by Uber to support their need for faster deep learning model training across many engineering teams. It is part of a growing ecosystem of approaches to distributed training, including for example, Distributed TensorFlow. Uber decided to develop a solution that utilised MPI for distributed process communication, and the NVIDIA Collective Communications Library (NCCL) for its highly optimised implementation of reductions across distributed processes and nodes. The resulting Horovod package delivers on its promise to scale deep learning model training across multiple GPUs and multiple nodes, with only minor code modification and intuitive debugging.


#### Horovod's MPI Roots

Horovod's connection to MPI runs deep, and for programmers familiar with MPI programming, much of what you program to distribute model training with Horovod will feel very familiar. For those unfamiliar with MPI programming, a brief discussion of some of the conventions and considerations required when distributing processes with Horovod, or MPI, is worthwhile. Horovod, as with MPI, strictly follows the Single-Program Multiple-Data (SPMD) paradigm where we implement the instruction flow of multiple processes in the same file/program. Because multiple processes are executing code in parallel, we have to take care about race conditions and also the synchronisation of participating processes.Horovod assigns a unique numerical ID or rank (an MPI concept) to each process executing the program. This rank can be accessed programmatically. As you will see below, when writing Horovod code, by identifying a process's rank programmatically in the code, we can take steps such as:

- Pin that process to its own exclusive GPU.
- Utilise a single rank for broadcasting values that need to be used uniformly by all ranks.
- Utilise a single rank for collecting and/or reducing values produced by all ranks.
- Utilise a single rank for logging or writing to disk.


To use Horovod with Tensorflow, we would need to make some modifications. The modifications can be listed as follows :

1. Initialise Horovod  

```python
hvd.init()
```

2. Pin each GPU to a single process.

```python
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
```

3. Wrap the optimiser in Horovod Distributed optimiser. The distributed optimiser delegates gradient computation to the original optimiser, averages gradients using `all-reduce` or `all-gather`, and then applies those averaged gradients. 

```python 
hvd.DistributedOptimizer
```


4. Broadcast the initial variable states from rank 0 to all other processes.

This is necessary to ensure consistent initialisation of all workers when training is started with random weights or restored from a checkpoint.

This can be done by using the `hvd.broadcast_variables` method after models and optimisers have been initialised.

5. Modify your code to save checkpoints only on worker 0 to prevent other workers from corrupting them.


Let us now go over the modifications using a test code. 

**Note that the below code will run as a single process and will only use one GPU , this is used for explanations and a description on how to write it for multi-GPUs is given at the end**

```python

#Import Necessary libraries
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import time
import os
import sys
# Import Horovod
import horovod.tensorflow.keras as hvd
# 1. Horovod: initialize Horovod.
hvd.init()

```

With the typical setup of one GPU per process, setting this to `local_rank`. The first process on the server will be allocated the first GPU, the second process will be allocated the second GPU, and so forth.

```python
# 2. Horovod: pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

    
# Load dataset and batching.
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data(path='mnist-%d.npz' % hvd.rank())

dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images[..., tf.newaxis] / 255.0, tf.float32),
             tf.cast(mnist_labels, tf.int64))
)
BATCH_SIZE_PER_REPLCIA = 1024
dataset = dataset.repeat().shuffle(10000).batch(BATCH_SIZE_PER_REPLCIA)

```

Now, each process will have a copy of the model, we then wrap the Optimizer with `hvd.DistributedOptimizer`

```python

# Building the model
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Use Adam optimizer for training 
opt = tf.optimizers.Adam()

# 3. Horovod: Wrap optimizer with DistributedOptimizer.
opt = hvd.DistributedOptimizer(opt, backward_passes_per_step=1, average_aggregated_gradients=True)

# Horovod: Specify `experimental_run_tf_function=False` to ensure TensorFlow
# uses hvd.DistributedOptimizer() to compute gradients.
mnist_model.compile(loss=tf.losses.SparseCategoricalCrossentropy(),
                    optimizer=opt,
                    metrics=['accuracy'],
                    experimental_run_tf_function=False)

```

#### Callbacks 

We define some necessary callbacks, they are : 

1. `hvd.callbacks.BroadcastGlobalVariablesCallback(0)` :  This is necessary to ensure consistent initialization of all workers when training is started with random weights or restored from a checkpoint.

2. `hvd.callbacks.MetricAverageCallback()` : Since we are not validating the full dataset on each worker anymore, each worker will have different validation results. To improve validation metric quality and reduce variance, we will average validation results among all workers.

3. `Throughput()` : We define the same callback used in the Tensorflow version. This callback gives us the value of the throughput in `Images/sec` to better understand the throughput of the system.

We also make sure to set the `verbose` parameter in `model.fit()` to ensure only one worker prints the results as all workers have identical results. We then fit the dataset, note that we have to define the `steps_per_epoch` parameter. 

```python

class Throughput(tf.keras.callbacks.Callback):
    def __init__(self, total_images=0):
        self.total_images = total_images
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
    def on_epoch_end(self, epoch, logs=None):
        if hvd.rank() == 0 :
            epoch_time = time.time() - self.epoch_start_time
            print('Epoch time : {}'.format(epoch_time))
            images_per_sec = round(self.total_images / epoch_time, 2)
            print('Images/sec: {}'.format(images_per_sec))

callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
    # Horovod: average metrics among workers at the end of every epoch.
    hvd.callbacks.MetricAverageCallback(),
    # Callback to calculate Throughput
    Throughput(total_images=len(mnist_labels))
]


# Horovod: write logs on worker 0.
verbose = 1 if hvd.rank() == 0 else 0

# Train the model.
# Horovod: adjust number of steps based on number of GPUs.
mnist_model.fit(dataset, steps_per_epoch=len(mnist_labels) // (BATCH_SIZE_PER_REPLCIA*hvd.size()), callbacks=callbacks, epochs=8, verbose=verbose)
```

Kindly restart the kernel or run the following cell to free up GPU memory before procedding further.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True) 

Like we mentioned earlier, the above cells would run as a single process and thus not utilise multiple GPUs. Now to launch multiple process, we need to use `horovodrun` command which inturn invokes the [mpirun](#https://www.open-mpi.org/doc/v4.0/man1/mpirun.1.php) command with certain optimisations.

To run on a machine with 4 GPUs:

```bash
$ horovodrun -np 4 python train.py
```

To run on 4 machines with 4 GPUs each:

```bash
$ horovodrun -np 16 -H server1:4,server2:4,server3:4,server4:4 python train.py
```

Let's run it with multiple-gpus by setting the `-np` flag , the `-np` flag sets the number of copies of the program that we want to run.

In [6]:
COMMAND = SINGULARITY_RUN + ' horovodrun -np 4 --mpi-args="--oversubscribe" python3 ../source_code/N3/cnn_fmnist.py --batch-size=8192 2> /dev/null'
!echo $COMMAND > command && srun --partition=gpu -n1 --gres=gpu:4 /bin/bash ./command

[1,0]<stdout>:Epoch 1/15
[1,0]<stdout>:1/1 - 29s - loss: 2.3436 - accuracy: 0.0801
[1,0]<stdout>:Epoch time : 29.148485898971558
[1,0]<stdout>:Images/sec: 2058.43
[1,0]<stdout>:Epoch 2/15
[1,0]<stdout>:1/1 - 2s - loss: 2.1719 - accuracy: 0.2725
[1,0]<stdout>:Epoch time : 2.3856217861175537
[1,0]<stdout>:Images/sec: 25150.68
[1,0]<stdout>:Epoch 3/15
[1,0]<stdout>:1/1 - 1s - loss: 1.9499 - accuracy: 0.5011
[1,0]<stdout>:Epoch time : 0.9173963069915771
[1,0]<stdout>:Images/sec: 65402.49
[1,0]<stdout>:Epoch 4/15
[1,0]<stdout>:1/1 - 1s - loss: 1.7046 - accuracy: 0.5522
[1,0]<stdout>:Epoch time : 0.8427751064300537
[1,0]<stdout>:Images/sec: 71193.37
[1,0]<stdout>:Epoch 5/15
[1,0]<stdout>:1/1 - 0s - loss: 1.4648 - accuracy: 0.5952
[1,0]<stdout>:Epoch time : 0.2723817825317383
[1,0]<stdout>:Images/sec: 220279.05
[1,0]<stdout>:Epoch 6/15
[1,0]<stdout>:1/1 - 0s - loss: 1.2454 - accuracy: 0.6304
[1,0]<stdout>:Epoch time : 0.17676448822021484
[1,0]<stdout>:Images/sec: 339434.69
[1,0]<stdout>:Epoch

### Result and scaling efficiency


#### Scaling efficiency

Let's look at the different results that obtained and calculate the scaling efficiency.

<br/>
<center> <b>A100</b> </center> 

|#GPUs |Samples/sec|Scaling efficiency|
|-|-|-|
|1| | |
|2| | |
|4| | |

Fill in increase throughput that was achieved with a scaling efficiency. Observe the loss and accuracy that were recorded for single GPU versus multiple GPU. 


`The increase in convergence time is noticed when training with larger batch sizes (when we scale across GPUs, we create a large batch that has a size multiplied by the number of GPUs). We shall discuss reasons for this and some techniques that can be used for faster convergence in the next notebook.`

***

## Licensing

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="2.1.System-Topology.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 33%; text-align: center;">
        <a href="1.Introduction-to-Distributed-Deep-Learning.ipynb">1</a>
        <a href="2.1.System-Topology.ipynb">2</a>
        <a >3</a>
        <a href="4.Convergence.ipynb">4</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="4.Convergence.ipynb">Next Notebook</a></span>
</div>

<p> <center> <a href="../Start_Here.ipynb">Home Page</a> </center> </p>

